In [65]:
import praw
import pandas as pd
from datetime import datetime as dt
from praw.models import MoreComments
import nltk
import nltk
nltk.download('vader_lexicon')
import nltk.sentiment

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Svensson/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Reddit API authentication:

In [5]:
reddit = praw.Reddit(client_id='BxrF2g9d3egQ4g', \
                     client_secret='T-pkCmWCBxLjVlNZk88OjvcH7sk', \
                     user_agent='SDS2019')

## Download comments for movies on reddit with timestamp

In [58]:
subreddit = reddit.subreddit('movies')
movie = subreddit.search('Once upon a time in Hollywood')


def get_reddit_comments(movie_title, testing = True):
    comments = []
    subid = []
    treeid = []
    date = []
    
    searchresults = subreddit.search(movie_title)
    i = 0
    for subidx, submission in enumerate(searchresults):
        
        if testing:
            if i > 5:
                break
            i += 1
            
        try:
            comment_tree = submission.comments.list()
            for treeidx, top_lvl_comment in enumerate(comment_tree):
                
                # Erstat disse to linjer hvis i vil have 
                # data ud af "morecomments" objekterne
                if isinstance(top_lvl_comment, MoreComments):
                    continue
                
                subid.append(subidx)
                comments.append(top_lvl_comment.body)
                treeid.append(treeidx)
                date.append(top_lvl_comment.created_utc)
                
        except:
            pass
    df = pd.DataFrame({'submission': subid, 'comment_tree': treeid, 'comment': comments, 'date': date})
    df['date'] = pd.to_datetime(df['date'], unit = 's')
    return df

In [59]:
once_upon = get_reddit_comments('once upon a time in Hollywood', True)

In [80]:
#print(once_upon)
pd.options.display.max_colwidth = 3000
once_upon['comment'].iloc[[3]]

3    That scene at the ranch when Cliff was going in to check on George Spahn was so fucking tense. Even the music from the TV was foreboding as hell.\n\nAnd holy shit, I didn't even realize that was Dakota Fanning. Totally didn't recognize her.
Name: comment, dtype: object

## Linguistic analysis of commments:
Output --> four mean scores

In [68]:
vader = nltk.sentiment.vader.SentimentIntensityAnalyzer()

def sentiment(movie_df)

    vader_df = pd.DataFrame(list(once_upon['comment'].apply(vader.polarity_scores)))
    vader_mean = 


,compound,neg,neu,pos
0,0.2244,0.106,0.749,0.145
1,0.0000,0.000,1.000,0.000
2,0.0000,0.000,1.000,0.000
3,-0.9036,0.218,0.782,0.000
4,0.6077,0.000,0.558,0.442
5,0.8524,0.000,0.632,0.368
6,0.4404,0.000,0.508,0.492
7,0.9169,0.000,0.719,0.281
8,0.4927,0.000,0.738,0.262
9,0.0772,0.000,0.947,0.053
